In [10]:
import cv2
import numpy as np
import os
import json
import ast

In [11]:
pixel_to_um = 0.1155
pixel_to_m = pixel_to_um * 1e-6

In [12]:
# Make a list of all folders in a folder
def get_folder_list(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [20]:
def get_BB(folder, set_list):
    bbox_dict = {}
    for s in set_list:
        seq = cv2.VideoCapture(f"{folder}/{s}/Image#%04d.jpg")
        if not seq.isOpened():
            print('error reading image sequence')
            break
        valid, frame = seq.read()
        if not valid:
            print('error reading image')
            break
        bbox = cv2.selectROIs("Select Rois",frame)
        print(bbox)
        bbox = [tuple(x) for x in bbox]
        bbox_dict[s] = bbox

    cv2.destroyAllWindows()
    return bbox_dict

In [21]:
def track(folder, tracker, bbox, output_file):
    '''
    Track an object initially denoted by a bounding box (bbox)
    and write out the location in an output file
    '''
    seq = cv2.VideoCapture(f"{folder}/Image#%04d.jpg")
    if not seq.isOpened():
        print('error reading image sequence')
    valid, frame = seq.read()
    if not valid:
        print('error reading image')

    # Initialize tracker with first frame and bounding box
    valid = tracker.init(frame, bbox)

    data = []
    while True:
        # Read a new frame
        valid, frame = seq.read()
        if not valid:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        valid, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if valid:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        x = [timer]
        x.extend(bbox)
        # data
        data.append(np.array(x))
        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):  # if press SPACE bar
            break
    
    # Save data
    np.savetxt(output_file, data, delimiter=',')
    cv2.destroyAllWindows()

In [22]:
def dict_to_json(dict, filename):
    '''Saves a dictionary to a json file'''
    with open(filename, 'w') as f:
        json.dump(dict, f)

def json_to_dict(filename):
    '''Loads a json file into a dictionary'''
    with open(filename) as f:
        return json.load(f)


In [23]:
def dict_to_txt(dict, filename):
    '''Saves a dictionary to a txt file'''
    with open(filename, 'w') as f:
        for key in dict:
            f.write(f"{key}:{dict[key]}\n")

def txt_to_dict(filename):
    '''Converts a txt file into a dictionary'''
    with open(filename) as f:
        return {line.split(':')[0]: ast.literal_eval(line.split(':')[1][:-1]) for line in f}

# Track the Blobs

In [24]:
# Pre-processing
set_list = get_folder_list('data')
bbox_dict = get_BB('data', set_list)
dict_to_txt(bbox_dict, 'bbox_dict.txt')

[[325 452  46  40]
 [836 332  38  27]
 [965  99  38  34]
 [862 636  45  41]]
[[ 771  224   31   31]
 [1083  202   23   22]
 [ 913  400   43   34]
 [ 934  573   38   40]]
[[685 365  42  30]
 [663 523  56  43]
 [ 71 557  45  36]]
[[111 556  39  39]
 [491 391  31  27]
 [636 196  55  45]]
[[ 670  345   44   38]
 [1105  232   42   43]
 [1027  517   39   38]]
[[468 541  36  28]
 [601 239  28  29]]
[[396 319  33  32]
 [911 435  40  41]]
[[689 230  27  31]
 [985 261  33  29]]
[[554  91  54  42]
 [360 681  41  31]]
[[704 314  24  24]
 [603 107  23  21]
 [694 651  20  24]]
[[540 183  52  40]
 [967 540  45  36]]
[[ 61 316  33  34]
 [779 186  25  22]]
[[ 540  548   48   31]
 [ 388  130   45   32]
 [ 904  211   43   29]
 [1126  589   50   33]]
[[269 415  38  28]
 [353 331  31  32]
 [643 225  36  30]
 [824 142  27  24]
 [668 667  31  31]]
[[ 600  342   43   38]
 [ 227  302   41   32]
 [ 961  112   44   38]
 [1056  609   45   40]]
[[ 458  167   32   32]
 [ 848  382   33   33]
 [1037  303   38   36]
 

In [25]:
# Tracking
bbox_dict = txt_to_dict('bbox_dict.txt')
tracker = cv2.TrackerCSRT_create()

for s in set_list:
    i = 0
    for bbox in bbox_dict[s]:
        print(s, i, bbox)
        tracker = cv2.TrackerCSRT_create()
        output = track(f"data/{s}", tracker, bbox, f"output/{s}-{i}.txt")
        i += 1

Set10a 0 (325, 452, 46, 40)
Set10a 1 (836, 332, 38, 27)
Set10a 2 (965, 99, 38, 34)
Set10a 3 (862, 636, 45, 41)
Set1a 0 (771, 224, 31, 31)
Set1a 1 (1083, 202, 23, 22)
Set1a 2 (913, 400, 43, 34)
Set1a 3 (934, 573, 38, 40)
set1b 0 (685, 365, 42, 30)
set1b 1 (663, 523, 56, 43)
set1b 2 (71, 557, 45, 36)
Set2a 0 (111, 556, 39, 39)
Set2a 1 (491, 391, 31, 27)
Set2a 2 (636, 196, 55, 45)
set2b 0 (670, 345, 44, 38)
set2b 1 (1105, 232, 42, 43)
set2b 2 (1027, 517, 39, 38)
Set3a 0 (468, 541, 36, 28)
Set3a 1 (601, 239, 28, 29)
set3b 0 (396, 319, 33, 32)
set3b 1 (911, 435, 40, 41)
Set4a 0 (689, 230, 27, 31)
Set4a 1 (985, 261, 33, 29)
set4b 0 (554, 91, 54, 42)
set4b 1 (360, 681, 41, 31)
Set5a 0 (704, 314, 24, 24)
Set5a 1 (603, 107, 23, 21)
Set5a 2 (694, 651, 20, 24)
set5b 0 (540, 183, 52, 40)
set5b 1 (967, 540, 45, 36)
Set6a 0 (61, 316, 33, 34)
Set6a 1 (779, 186, 25, 22)
set6b 0 (540, 548, 48, 31)
set6b 1 (388, 130, 45, 32)
set6b 2 (904, 211, 43, 29)
set6b 3 (1126, 589, 50, 33)
Set7a 0 (269, 415, 38, 2